# FIR design in the frequency domain using numerical optimisation
This script demonstrates how, in practice, FIR filters can be designed using numerical optimisation techniques.  The resulting filters meet the design criteria specified only if there are sufficient taps to accommodate the level of stopband rejection whilst meeting transition band criteria.

### Preamble
Start by importing the Python libraries that we will require

In [ ]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt

And define a function that will return true if running in a Jupyter Notebook

In [ ]:
def is_jupyter():
    """Return true if running in a Jupyter Notebook"""
    try:
        if get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
            return True
        else:
            return False
    except: 
        return False

### Specify filter criteria
Here the filter criteria are specified.

For the Remez exchange algorithm implementation in Scipy, the band edges (lower and upper) are specified in the array, <code>bands</code>, with a single entry specifying the gain over that band in <code>desired</code>.  So <code>desired</code> is half of the length of <code>bands</code>.

For the Least Squares approach, a variable gain can be specified for each band by specifying the gain at the start and also at the end of each band, so <code>desired</code> is the same length as <code>bands</code>.

Weightings of the importance of the different bands can be specified in <code>weights</code> which can be used to enforce stricter, or looser, adherence to the specified criteria.

Parameter | Meaning
--------- | -------
<code>numtaps</code> | The number of taps in the FIR filter. <code>numtaps</code> must be odd for fftls.
<code>bands</code> | A monotonic nondecreasing sequence containing the band edges in normalised frequency. All elements must be non-negative and less than or equal 0.5
<code>desired_remez</code> | A sequence half the size of bands containing the desired gain of each band
<code>desired_ls</code> | A sequence the same size as bands containing the desired gain at the start and end point of each band
<code>weight</code>  | A sequence half the size of bands containing the relative weighting of each band
<code>fs</code> | Sampling frequency

In [ ]:
numtaps = 65
bands = [0, 0.1, 0.12, 0.5]
desired_remez = [1, 0]
desired_ls = [1, 1, 0, 0]
weight = [1, 1]
fs = 1

### Plotting Functions
Here we define the plotting functions that we will use for producing the outputs.

In [ ]:
def plot_stem(x, y, name, xlim = [0, 63]):
    """
       Plot FIR frequency design
       INPUT: 
           xlim ([left, right]): set the x limits from let to right. Default: [0, 63].
           x       (array-like): The x-positions of the stems
           y       (array-like): The y-values of the stem heads.
           name        (string): The name used to save figure.
    """
    # Create the plot figure
    plt.figure(figsize=(12, 6))
    
    # Enlarge figure label and axis size
    plt.rcParams.update({'font.size': 16})
    
    # stem plot  
    (markerLines, stemLines, baseLines) = plt.stem(x, y, use_line_collection=True)
    plt.setp(baseLines, color = 'black', linewidth=1) 
    plt.setp(stemLines, linewidth=1) 
    markerLines.set_markersize(8)
    markerLines.set_markerfacecolor('none')
    
    # Tidy up the plot to control axes sizes and labels
    plt.xlabel('n')
    plt.ylabel('h(n)')
    plt.xlim(xlim)
    
    # Save figure in python or ipython system
    if not is_jupyter():
        plt.tight_layout()
        plt.savefig(name)

In [ ]:
def plot_freq(x, y, name, xlims = [0, 0.5], ylims = [-80, 10]):
    """
       Plot magnitude in dB scale of normalised frequency 
    """
    # Create the plot figure
    plt.figure(figsize=(12, 6))
    
    # Enlarge figure label and axis size
    plt.rcParams.update({'font.size': 16})
    
    # Plot the frequency
    ax = plt.gca()
    ax.plot(x, y)
    
    # Tidy up the plot to control axes sizes and labels
    plt.xlabel('Normalised frequency')
    plt.ylabel('Magnitude (dB)')
    plt.xlim(xlims)
    plt.ylim(ylims)
    plt.xticks(np.linspace(0, 0.5, 11))
    
    # Save figure in python or ipython system
    if not is_jupyter():
        plt.tight_layout()
        plt.savefig(name)

### Remez exchange algorithm

The first approach is to generate an impulse response using the Remez exchange algorithm

In [ ]:
# Generate impulse response
h1 = signal.remez(numtaps, bands, desired_remez, weight=weight, fs=fs)

# Zero padd this for plotting
z = np.zeros(4096)
z[0 : h1.size] = h1

n = np.arange(0,h1.size)

# Generate frequency index
f = np.multiply((fs/4096), np.arange(0, 4096))

# Calculate the transfer function corresponding to the impulse response
H = 20 * np.log10(abs(np.fft.fft(z)))

Plot the filter impulse response

In [ ]:
plot_stem(x = n, y = h1, name = 'FIR_freq_design_remez_impulse.pdf')

Plot the transfer function, along with the original samples that were used to generate the impulse response.

In [ ]:
plot_freq(x = f, y= H, name = 'FIR_freq_design_remez_freq.pdf')

### Least Squares approach

The second approach, the Least Squares algorithm, minimises the difference between the desired gain and the actual gain.

In [ ]:
# Generate impulse response
h2 = signal.firls(numtaps, bands, desired_ls, weight=weight, fs=fs)

# Zero padd this for plotting
z[0 : h2.size] = h2

n = np.arange(0,h2.size)

# Generate frequency index
f = np.multiply((fs/4096), np.arange(0, 4096))

# Calculate the transfer function corresponding to the impulse response
H = 20 * np.log10(abs(np.fft.fft(z)))

Plot the impulse response

In [ ]:
plot_stem(x = n, y = h2, name = 'FIR_freq_design_ls_impulse.pdf')

Plot the transfer function

In [ ]:
plot_freq(x = f, y = H, name = 'FIR_freq_design_ls_freq.pdf')

© The University of Edinburgh: Produced by D. Laurenson, School of Engineering. Initial code conversion by Xing Zixiao.